In [1]:
import numpy as np
import pandas as pd
import ast
movies = pd.read_csv('tmdb_5000_movies.csv')
credits= pd.read_csv('tmdb_5000_credits.csv')
movies = movies.merge(credits, on='title')
movies = movies[['movie_id','title','genres','overview','keywords','cast','crew']]

In [2]:
movies.dropna(inplace = True)
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
def convertcast(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L
def convertdir(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L
movies['overview']=   movies['overview'].apply(lambda x:x.split())
movies['genres']  = movies['genres'].apply(convert)
movies['keywords']= movies['keywords'].apply(convert)
movies['cast']    = movies['cast'].apply(convertcast)
movies['crew']    = movies['crew'].apply(convertdir)
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1
movies['cast']    = movies['cast'].apply(collapse)
movies['crew']    = movies['crew'].apply(collapse)
movies['genres']  = movies['genres'].apply(collapse)
movies['keywords']= movies['keywords'].apply(collapse)
movies['tags']    = movies['genres']+movies['keywords']+movies['cast']+movies['crew'] 
new_df            = movies.drop(columns=['overview','genres','keywords','cast','crew'])
new_df['tags']    = new_df['tags'].apply(lambda x:" ".join(x))
new_df['tags']    = new_df['tags'].apply(lambda x:x.lower())
new_df.head()

,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()
vector.shape
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [12]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [13]:
recommend('Avatar')

Star Trek Into Darkness
Megaforce
Jupiter Ascending
The Lovers
Aliens
